# 목차
1. 필요한 라이브러리 임포트 및 설정
2. 슬라이딩 윈도우 함수 정의
3. 특성 추출 함수 정의
4. 데이터 증강 함수 정의
5. MHealth 데이터 로드 및 전처리
- 5.1 데이터 로드
- 5.2 데이터 결합
- 5.3 슬라이딩 윈도우 적용 및 특성 추출

6. MergeData.csv 데이터 로드 및 전처리
- 6.1 데이터 로드 및 슬라이딩 윈도우 적용
- 6.2 데이터 결합

7. 레이블이 있는 데이터와 없는 데이터 
- 7.1 데이터 결합
- 7.2 데이터 증강 및 스케일링

8. 데이터셋 분할 및 시퀀스 생성
- 8.1 레이블이 있는 데이터와 없는 데이터 분리
- 8.2 학습 및 검증 데이터 분할
- 8.3 시퀀스 생성

9. 데이터셋 및 데이터로더 생성
10. CNN-LSTM 모델 정의
11. 하이퍼파라미터 튜닝
- 11.1 Objective 함수 정의
- 11.2 스터디 객체 생성 및 최적화 수행

12. 모델 학습
13. 모델 평가 및 시각화
14. MergeData.csv 데이터에 대한 예측 및 레이블 추가
- 14.1 MergeData.csv 데이터 예측
- 14.2 예측된 레이블 추가 및 저장
- 14.3 예측 결과 시각화

15. 마무리

# 1. 필요한 라이브러리 임포트 및 설정

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # 진행 상황 표시

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pywt
import optuna  # 하이퍼파라미터 튜닝을 위한 라이브러리

import warnings
import logging

warnings.filterwarnings('ignore')  # 경고 메시지 무시

# 로깅 설정
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler("training.log"),
                        logging.StreamHandler()
                    ])
logger = logging.getLogger()

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"사용 중인 디바이스: {device}")
if device.type == 'cuda':
    logger.info(f"GPU 이름: {torch.cuda.get_device_name(0)}")

2024-10-18 15:03:26,798 - INFO - 사용 중인 디바이스: cuda
2024-10-18 15:03:26,805 - INFO - GPU 이름: NVIDIA GeForce GTX 1650


# 2. 슬라이딩 윈도우 함수 정의

In [2]:
def sliding_window(data, window_size, step_size):
    """
    슬라이딩 윈도우를 적용하여 데이터를 윈도우로 분할합니다.

    Parameters:
        data (numpy array): 입력 데이터 (samples, channels).
        window_size (int): 윈도우 크기.
        step_size (int): 윈도우 이동 크기.

    Returns:
        numpy array: 윈도우로 분할된 데이터 (num_windows, window_size, channels).
    """
    
    num_samples = data.shape[0]

    if num_samples < window_size:
        return np.array([])  # 빈 배열 반환

    windows = []
    
    for start in range(0, num_samples - window_size + 1, step_size):
        end = start + window_size
        window = data[start:end, :]
        windows.append(window)
        
    return np.array(windows)

# 3. 특성 추출 함수 정의

In [3]:
def extract_features(windows):
    """
    각 윈도우에 대해 특성을 추출합니다.

    Parameters:
        windows (numpy array): 윈도우로 분할된 데이터 (num_windows, window_size, channels).

    Returns:
        numpy array: 추출된 특성 (num_windows, num_features).
    """
    
    features = []

    logger.info("Extracting features from windows...")
    
    for window in tqdm(windows, desc="Extracting Features"):
        window_features = []

        if window.ndim == 1:
            window = window[:, np.newaxis]

        num_channels = window.shape[1]

        for channel in range(num_channels):
            signal = window[:, channel]

            # 기본 통계량
            mean = np.mean(signal)
            std = np.std(signal)
            skewness = stats.skew(signal)
            kurtosis = stats.kurtosis(signal)
            rms = np.sqrt(np.mean(signal**2))

            # FFT
            fft_coeffs = np.fft.rfft(signal)
            fft_mean = np.mean(np.abs(fft_coeffs))
            fft_std = np.std(np.abs(fft_coeffs))

            # 웨이블릿 변환 후 에너지
            coeffs = pywt.wavedec(signal, 'db1', level=3)
            energy = sum([np.sum(c**2) for c in coeffs])

            # 특성 벡터에 추가
            window_features.extend([mean, std, skewness, kurtosis, rms, fft_mean, fft_std, energy])

        features.append(window_features)
    return np.array(features)

# 4. 데이터 증강 함수 정의

In [4]:
def augment_data(X, y):
    """
    데이터 증강을 위해 다양한 기법을 적용합니다.

    Parameters:
        X (numpy array): 입력 데이터.
        y (numpy array): 레이블.

    Returns:
        tuple: 증강된 데이터와 레이블.
    """
    
    # 가우시안 노이즈 추가
    noise = np.random.normal(0, 0.01, X.shape)
    X_noise = X + noise

    # 시프트 (Shift) 증강
    shift = np.random.randint(-5, 5, size=X.shape)
    X_shift = X + shift

    # 스케일링 (Scaling) 증강
    scaling_factor = np.random.uniform(0.9, 1.1, size=(X.shape[0], X.shape[1]))
    X_scaled = X * scaling_factor

    # 증강된 데이터 결합
    X_augmented = np.vstack((X_noise, X_shift, X_scaled))
    y_augmented = np.hstack((y, y, y))

    return X_augmented, y_augmented

# 5. MHealth 데이터 로드 및 전처리
***
## 5.1 데이터 로드

In [5]:
# 데이터 파일 경로 설정 (실제 데이터 경로로 수정하세요)
data_files = glob.glob('C:/AI/data/MHEALTHDATASET/mHealth_subject*.log')  # 실제 경로로 변경

# 데이터와 레이블을 저장할 리스트 초기화
X_list = []
y_list = []

logger.info("Loading MHealth dataset...")
for file in tqdm(data_files, desc="Loading MHealth Files"):
    data = pd.read_csv(file, sep='\s+', header=None)
    logger.info(f"파일명: {file}")
    logger.info(f"data.shape: {data.shape}")

    # 데이터의 열 수가 24개인지 확인
    if data.shape[1] != 24:
        logger.warning(f"데이터의 열 수가 24가 아닙니다. 파일을 확인하세요: {file}")
        continue  # 다음 파일로 넘어감

    # 우측 하완 센서의 자이로스코프 데이터 추출 (열 18~20)
    right_arm_gyro = data.iloc[:, 17:20].values   # 열 18~20 (Python 인덱스는 0부터 시작)
    logger.info(f"right_arm_gyro.shape: {right_arm_gyro.shape}")

    # 레이블 추출 (열 24)
    labels = data.iloc[:, 23].values
    logger.info(f"labels.shape: {labels.shape}")

    # null 클래스(레이블 0)는 제외
    valid_indices = labels != 0
    sensor_data = right_arm_gyro[valid_indices, :]  # 모든 열 선택
    labels = labels[valid_indices]

    # 데이터와 레이블 저장
    X_list.append(sensor_data)
    y_list.append(labels)

logger.info("MHealth 데이터 로드 완료.")

2024-10-18 15:03:26,938 - INFO - Loading MHealth dataset...
Loading MHealth Files:   0%|                                  | 0/10 [00:00<?, ?it/s]2024-10-18 15:03:28,008 - INFO - 파일명: C:/AI/data/MHEALTHDATASET\mHealth_subject1.log
2024-10-18 15:03:28,009 - INFO - data.shape: (161280, 24)
2024-10-18 15:03:28,012 - INFO - right_arm_gyro.shape: (161280, 3)
2024-10-18 15:03:28,013 - INFO - labels.shape: (161280,)
Loading MHealth Files:  10%|██▌                       | 1/10 [00:01<00:09,  1.07s/it]2024-10-18 15:03:28,663 - INFO - 파일명: C:/AI/data/MHEALTHDATASET\mHealth_subject10.log
2024-10-18 15:03:28,664 - INFO - data.shape: (98304, 24)
2024-10-18 15:03:28,666 - INFO - right_arm_gyro.shape: (98304, 3)
2024-10-18 15:03:28,667 - INFO - labels.shape: (98304,)
Loading MHealth Files:  20%|█████▏                    | 2/10 [00:01<00:06,  1.21it/s]2024-10-18 15:03:29,583 - INFO - 파일명: C:/AI/data/MHEALTHDATASET\mHealth_subject2.log
2024-10-18 15:03:29,584 - INFO - data.shape: (130561, 24)
2024-10-18

# 5.2 데이터 결합

In [6]:
# 리스트를 배열로 변환
X_raw_mhealth = np.vstack(X_list)
y_raw_mhealth = np.hstack(y_list)

# 레이블을 0부터 시작하도록 조정
y_raw_mhealth = y_raw_mhealth - 1  # 레이블이 1부터 시작하므로 0부터 시작하도록 조정

logger.info(f"전체 MHealth 데이터 크기: {X_raw_mhealth.shape}")
logger.info(f"전체 MHealth 레이블 크기: {y_raw_mhealth.shape}")

2024-10-18 15:03:35,117 - INFO - 전체 MHealth 데이터 크기: (343195, 3)
2024-10-18 15:03:35,119 - INFO - 전체 MHealth 레이블 크기: (343195,)


# 5.3 슬라이딩 윈도우 적용 및 특성 추출

In [7]:
# 윈도우 설정
window_size = 128  # 샘플 수에 따라 조정 가능
step_size = 64

logger.info("Applying sliding window to MHealth data...")
# 슬라이딩 윈도우 적용
windows = sliding_window(X_raw_mhealth, window_size, step_size)

# 레이블 슬라이딩 윈도우 적용 (다수결을 통해 레이블 결정)
labels_windows = sliding_window(y_raw_mhealth.reshape(-1, 1), window_size, step_size)

logger.info("Assigning labels to each window based on majority vote...")
window_labels = [np.bincount(label_window.flatten().astype(int)).argmax() for label_window in tqdm(labels_windows, desc="Assigning Window Labels")]

# 특성 추출
logger.info("Extracting features from MHealth windows...")
X_features_mhealth = extract_features(windows)
y_labels_mhealth = np.array(window_labels)

logger.info(f"MHealth 데이터 특성 크기: {X_features_mhealth.shape}")
logger.info(f"MHealth 레이블 크기: {y_labels_mhealth.shape}")

2024-10-18 15:03:35,146 - INFO - Applying sliding window to MHealth data...
2024-10-18 15:03:35,160 - INFO - Assigning labels to each window based on majority vote...
Assigning Window Labels: 100%|███████████████| 5361/5361 [00:00<00:00, 297760.26it/s]
2024-10-18 15:03:35,182 - INFO - Extracting features from MHealth windows...
2024-10-18 15:03:35,182 - INFO - Extracting features from windows...
Extracting Features: 100%|██████████████████████| 5361/5361 [00:17<00:00, 298.41it/s]
2024-10-18 15:03:53,159 - INFO - MHealth 데이터 특성 크기: (5361, 24)
2024-10-18 15:03:53,160 - INFO - MHealth 레이블 크기: (5361,)


# 6. MergeData.csv 데이터 로드 및 전처리
***
## 6.1 데이터 로드 및 슬라이딩 윈도우 적용

In [ ]:
# MergeData.csv 파일 경로 설정
merge_data_path = 'C:/AI/data/MergeData.csv'

# 청크 크기 설정 (메모리 용량에 따라 조정)
chunk_size = 100000  # 한 번에 읽어올 행의 수

# 데이터와 레이블을 저장할 리스트 초기화
X_list_merge = []

# 컬럼 이름 지정
column_names = ['VitalDate',
                'WorkDate',
                'UserCode',
                'HeartBeat',
                'Temperature',
                'OutsideTemperature',
                'Latitude',
                'Longitude',
                'X',
                'Y',
                'Z']

logger.info("Loading and processing MergeData.csv...")
# 데이터 로드 및 전처리
reader = pd.read_csv(merge_data_path,
                     chunksize=chunk_size,
                     names=column_names,
                     header=0)

for chunk in tqdm(reader, desc='Processing MergeData Chunks'):
    # 자이로스코프 데이터 추출 (X, Y, Z 컬럼)
    gyro_data = chunk[['X', 'Y', 'Z']].values

    if gyro_data.shape[0] < window_size:
        continue  # 데이터 길이가 윈도우 크기보다 작으면 건너뜀

    # 슬라이딩 윈도우 적용
    windows = sliding_window(gyro_data,
                             window_size=window_size,
                             step_size=step_size)

    if windows.size == 0:
        continue

    # 특성 추출
    features = extract_features(windows)

    # 데이터 저장
    X_list_merge.append(features)

logger.info("MergeData.csv 데이터 처리 완료.")

2024-10-18 15:03:53,174 - INFO - Loading and processing MergeData.csv...
Processing MergeData Chunks: 0it [00:00, ?it/s]2024-10-18 15:03:53,313 - INFO - Extracting features from windows...

Extracting Features: 100%|██████████████████████| 1561/1561 [00:05<00:00, 295.48it/s]
Processing MergeData Chunks: 1it [00:05,  5.41s/it]2024-10-18 15:03:58,716 - INFO - Extracting features from windows...

Extracting Features: 100%|██████████████████████| 1561/1561 [00:05<00:00, 291.39it/s]
Processing MergeData Chunks: 2it [00:10,  5.45s/it]2024-10-18 15:04:04,194 - INFO - Extracting features from windows...

Extracting Features: 100%|██████████████████████| 1561/1561 [00:05<00:00, 295.70it/s]
Processing MergeData Chunks: 3it [00:16,  5.43s/it]2024-10-18 15:04:09,596 - INFO - Extracting features from windows...

Extracting Features: 100%|██████████████████████| 1561/1561 [00:05<00:00, 295.56it/s]
Processing MergeData Chunks: 4it [00:21,  5.42s/it]2024-10-18 15:04:14,991 - INFO - Extracting features

# 6.2 데이터 결합

In [ ]:
# 리스트를 배열로 변환
if X_list_merge:
    X_raw_merge = np.vstack(X_list_merge)
    logger.info(f"전체 MergeData 특성 데이터 크기: {X_raw_merge.shape}")
else:
    X_raw_merge = np.array([])
    logger.warning("MergeData.csv 데이터가 비어있습니다.")

# 7. 레이블이 있는 데이터와 없는 데이터 결합
***
## 7.1 데이터 결합

In [ ]:
# MHEALTH 데이터의 레이블은 그대로 유지
if X_raw_merge.size > 0:
    X_total = np.vstack((X_features_mhealth, X_raw_merge))
    # 레이블은 MHEALTH 데이터의 레이블과 MergeData의 레이블(없는 부분)을 결합
    y_total = np.hstack((y_labels_mhealth, np.full(X_raw_merge.shape[0], -1)))  # 레이블이 없는 부분은 -1로 표시
else:
    X_total = X_features_mhealth
    y_total = y_labels_mhealth

logger.info(f"결합된 데이터 크기: {X_total.shape}")
logger.info(f"결합된 레이블 크기: {y_total.shape}")

## 7.2 데이터 증강 및 스케일링

In [ ]:
logger.info("Applying data augmentation to labeled data...")
# 레이블이 있는 데이터에 대해서만 데이터 증강 적용
X_augmented, y_augmented = augment_data(X_features_mhealth, y_labels_mhealth)

# 원본 MHEALTH 데이터와 증강된 데이터 결합
X_mhealth_combined = np.vstack((X_features_mhealth, X_augmented))
y_mhealth_combined = np.hstack((y_labels_mhealth, y_augmented))

logger.info(f"증강된 MHEALTH 데이터 크기: {X_mhealth_combined.shape}")
logger.info(f"증강된 MHEALTH 레이블 크기: {y_mhealth_combined.shape}")

# 전체 데이터 결합 (MHEALTH 데이터와 MergeData 데이터)
if X_raw_merge.size > 0:
    X_total = np.vstack((X_mhealth_combined, X_raw_merge))
    y_total = np.hstack((y_mhealth_combined, np.full(X_raw_merge.shape[0], -1)))  # 레이블이 없는 부분은 -1로 표시
else:
    X_total = X_mhealth_combined
    y_total = y_mhealth_combined

logger.info(f"결합된 데이터 크기: {X_total.shape}")
logger.info(f"결합된 레이블 크기: {y_total.shape}")

# 스케일링
logger.info("Scaling the data...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_total)

# 스케일러 저장
joblib.dump(scaler, './scaler/scaler_total.pkl')
logger.info("스케일러를 'scaler_total.pkl'로 저장했습니다.")

# 8. 데이터셋 분할 및 시퀀스 생성
***
## 8.1 레이블이 있는 데이터와 없는 데이터 분리

In [ ]:
logger.info("Separating labeled and unlabeled data...")
# 레이블이 있는 데이터만 분리
X_labeled = X_scaled[y_total != -1]
y_labeled = y_total[y_total != -1].astype(int)

# 레이블이 없는 데이터만 분리
X_unlabeled = X_scaled[y_total == -1]

logger.info(f"레이블이 있는 데이터 크기: {X_labeled.shape}")
logger.info(f"레이블이 없는 데이터 크기: {X_unlabeled.shape}")

## 8.2 학습 및 검증 데이터 분할

In [ ]:
logger.info("Splitting labeled data into training and validation sets...")
# 레이블이 있는 데이터셋 분할
X_train, X_val, y_train, y_val = train_test_split(
    X_labeled,
    y_labeled,
    test_size=0.2,
    random_state=42,
    stratify=y_labeled
)

logger.info(f"학습 데이터 크기: {X_train.shape}")
logger.info(f"검증 데이터 크기: {X_val.shape}")

## 8.3 시퀀스 생성

In [ ]:
# 시퀀스 길이 설정
seq_len = 10

def create_sequences(features, labels, seq_len):
    """
    시퀀스 데이터를 생성합니다.

    Parameters:
        features (numpy array): 입력 특성 데이터.
        labels (numpy array): 레이블 데이터.
        seq_len (int): 시퀀스 길이.

    Returns:
        tuple: 시퀀스 데이터와 시퀀스 레이블.
    """
    
    sequences = []
    sequence_labels = []
    
    for i in range(len(features) - seq_len + 1):
        seq = features[i:i+seq_len]
        label = labels[i + seq_len - 1]  # 마지막 윈도우의 레이블을 시퀀스의 레이블로 사용
        sequences.append(seq)
        sequence_labels.append(label)
    
    return np.array(sequences), np.array(sequence_labels)

logger.info("Creating sequences for training data...")
# 학습 데이터에 시퀀스 생성
X_train_seq, y_train_seq = create_sequences(X_train, y_train, seq_len)
logger.info(f"학습 시퀀스 데이터 크기: {X_train_seq.shape}")
logger.info(f"학습 시퀀스 레이블 크기: {y_train_seq.shape}")

logger.info("Creating sequences for validation data...")
# 검증 데이터에 시퀀스 생성
X_val_seq, y_val_seq = create_sequences(X_val, y_val, seq_len)
logger.info(f"검증 시퀀스 데이터 크기: {X_val_seq.shape}")
logger.info(f"검증 시퀀스 레이블 크기: {y_val_seq.shape}")

# 9. 데이터셋 및 데이터로더 생성

In [ ]:
class ActivityDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
        
    def __len__(self):
        return len(self.y)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

logger.info("Creating Dataset and DataLoader...")
# 데이터셋 생성
train_dataset = ActivityDataset(X_train_seq, y_train_seq)
val_dataset = ActivityDataset(X_val_seq, y_val_seq)

# 데이터로더 생성
batch_size = 64
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0,  # Jupyter Notebook 환경에서는 0으로 설정
                          pin_memory=True)
val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=0,  # Jupyter Notebook 환경에서는 0으로 설정
                        pin_memory=True)

logger.info("Dataset and DataLoader 생성 완료.")

# 10. CNN-LSTM 모델 정의

In [ ]:
class CNNLSTMModel(nn.Module):
    def __init__(self, num_features, num_classes, dropout):
        super(CNNLSTMModel, self).__init__()
        self.num_features = num_features
        
        # Convolutional Layers
        self.conv1 = nn.Conv1d(in_channels=num_features,
                               out_channels=64,
                               kernel_size=3,
                               padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64,
                               out_channels=128,
                               kernel_size=3,
                               padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout_conv = nn.Dropout(dropout)
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size=128,
                            hidden_size=256,
                            num_layers=2,
                            batch_first=True,
                            dropout=dropout)
        
        # Fully Connected Layer
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        # x shape: (batch_size, seq_len, num_features)
        x = x.permute(0, 2, 1)  # (batch_size, num_features, seq_len)
        
        # Convolutional Layers
        x = torch.relu(self.bn1(self.conv1(x)))  # (batch_size, 64, seq_len)
        x = torch.relu(self.bn2(self.conv2(x)))  # (batch_size, 128, seq_len)
        x = self.pool(x)                        # (batch_size, 128, seq_len/2)
        x = self.dropout_conv(x)
        x = x.permute(0, 2, 1)                  # (batch_size, seq_len/2, 128)
        
        # LSTM Layer
        x, _ = self.lstm(x)                     # (batch_size, seq_len/2, 256)
        x = x[:, -1, :]                         # (batch_size, 256)
        
        # Fully Connected Layer
        x = self.fc(x)                          # (batch_size, num_classes)
        return x

# 모델 초기화
num_features = X_train_seq.shape[2]  # num_features는 특성 수 (24)
num_classes = len(np.unique(y_train_seq))
dropout = 0.5

model = CNNLSTMModel(num_features, 
                     num_classes,
                     dropout)
model.to(device)

logger.info("CNN-LSTM 모델 구조:")
logger.info(model)

# 11. 하이퍼파라미터 튜닝
***
## 11.1 Objective 함수 정의

In [ ]:
def objective(trial):
    """
    Optuna의 objective 함수 정의.

    Parameters:
        trial (optuna.trial.Trial): Optuna trial 객체.

    Returns:
        float: 검증 손실.
    """
    
    # 하이퍼파라미터 제안
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    hidden_size = trial.suggest_int('hidden_size', 128, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)

    # 모델 초기화
    model = CNNLSTMModel(num_features, num_classes, dropout)
    model.lstm.hidden_size = hidden_size
    model.lstm.num_layers = num_layers
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # 학습 설정
    num_epochs = 10
    best_val_loss = float('inf')
    patience = 3
    counter = 0

    for epoch in range(num_epochs):
        # 학습 모드
        model.train()
        total_loss = 0
        total_correct = 0

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == y_batch).sum().item()

        train_loss = total_loss / len(train_dataset)
        train_acc = total_correct / len(train_dataset)

        # 검증 모드
        model.eval()
        total_loss = 0
        total_correct = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                
                total_loss += loss.item() * X_batch.size(0)
                _, preds = torch.max(outputs, 1)
                total_correct += (preds == y_batch).sum().item()

        val_loss = total_loss / len(val_dataset)
        val_acc = total_correct / len(val_dataset)

        # 검증 손실이 개선되었는지 확인
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break

        # Optuna에 보고
        trial.report(val_loss, epoch)

        # 조기 종료 판단
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss  # 검증 손실을 최소화

## 11.2 스터디 객체 생성 및 최적화 수행

In [ ]:
logger.info("Starting hyperparameter tuning with Optuna...")
# 스터디 객체 생성
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, timeout=None)

# 최적의 하이퍼파라미터 출력
logger.info("Best hyperparameters: ")
for key, value in study.best_params.items():
    logger.info(f"  {key}: {value}")

# 12. 모델 학습

In [ ]:
# 최적의 하이퍼파라미터로 모델 초기화
best_dropout = study.best_params['dropout']
best_learning_rate = study.best_params['learning_rate']
best_hidden_size = study.best_params['hidden_size']
best_num_layers = study.best_params['num_layers']

logger.info(f"Initializing model with best hyperparameters: dropout={best_dropout}, learning_rate={best_learning_rate}, hidden_size={best_hidden_size}, num_layers={best_num_layers}")
model = CNNLSTMModel(num_features,
                     num_classes,
                     best_dropout)
model.lstm.hidden_size = best_hidden_size
model.lstm.num_layers = best_num_layers
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_learning_rate)

# 학습 설정
num_epochs = 200
patience = 5
best_val_loss = float('inf')
counter = 0

# 손실과 정확도를 저장할 리스트 초기화
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

logger.info("Starting training with optimized hyperparameters...")
for epoch in range(num_epochs):
    # 학습 모드
    model.train()
    total_loss = 0
    total_correct = 0
    
    # tqdm을 사용하여 배치별 진행 상황 시각화
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training", leave=False):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * X_batch.size(0)
        _, preds = torch.max(outputs, 1)
        total_correct += (preds == y_batch).sum().item()
    
    train_loss = total_loss / len(train_dataset)
    train_acc = total_correct / len(train_dataset)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    
    # 검증 모드
    model.eval()
    total_loss = 0
    total_correct = 0
    
    with torch.no_grad():
        for X_batch, y_batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation", leave=False):
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            total_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            total_correct += (preds == y_batch).sum().item()
    
    val_loss = total_loss / len(val_dataset)
    val_acc = total_correct / len(val_dataset)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        counter = 0
        logger.info(f"Epoch {epoch+1}: 검증 손실이 개선되어 모델을 저장했습니다. Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
    else:
        counter += 1
        logger.info(f"Epoch {epoch+1}: 검증 손실이 개선되지 않았습니다. Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%")
        if counter >= patience:
            logger.info("조기 종료 조건에 도달하여 학습을 종료합니다.")
            break
    
    # 에포크 결과 출력
    logger.info(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, "
                f"Train Acc: {train_acc*100:.2f}%, Val Loss: {val_loss:.4f}, "
                f"Val Acc: {val_acc*100:.2f}%")

# 13. 모델 평가 및 시각화
***
## 13.1. 학습 및 검증 손실, 정확도 곡선 그리기

In [ ]:
logger.info("Plotting loss and accuracy curves...")

plt.figure(figsize=(12, 5))

# 손실 곡선 그리기
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='orange')
plt.title('Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 정확도 곡선 그리기
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy', color='blue')
plt.plot(val_accuracies, label='Validation Accuracy', color='orange')
plt.title('Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

- 손실 곡선은 모델의 학습 및 검증 손실을 보여주며, 오버피팅 여부를 판단하는 데 도움이 됩니다.
- 정확도 곡선은 모델의 학습 및 검증 정확도를 보여줍니다.

## 13.2. 혼동 행렬 시각화 및 분류 보고서 출력

In [ ]:
logger.info("Generating confusion matrix and classification report...")

# 전체 검증 데이터에 대한 예측 수행
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in tqdm(val_loader, desc="Generating Predictions"):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# 혼동 행렬 계산
cm = confusion_matrix(all_labels, all_preds)

# 클래스 이름 정의 (레이블 인덱스와 매칭)
class_names = ['WALKING',
               'WALKING_UPSTAIRS',
               'WALKING_DOWNSTAIRS',
               'SITTING',
               'STANDING',
               'LAYING']

# 혼동 행렬 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

# 분류 보고서 출력
logger.info("Classification Report:")
report = classification_report(all_labels, all_preds, target_names=class_names)
logger.info(f"\n{report}")
print("Classification Report:")
print(report)

# 14. MergeData.csv 데이터에 대한 예측 및 레이블 추가
***
## 14.1. MergeData.csv 데이터 예측

In [ ]:
# 저장된 모델 및 스케일러 로드
logger.info("Loading the best model and scaler for prediction...")
model = CNNLSTMModel(num_features,
                     num_classes,
                     best_dropout)
model.lstm.hidden_size = best_hidden_size
model.lstm.num_layers = best_num_layers
model.load_state_dict(torch.load('./model/best_model.pth'))
model.to(device)
model.eval()
logger.info("모델과 스케일러 로드 완료.")

scaler = joblib.load('./scaler/scaler_total.pkl')
pca = joblib.load('scaler_total.pkl')  # 이미 스케일링된 데이터이므로 PCA는 필요 없습니다.

# 클래스 이름 정의
class_names = ['WALKING',
               'WALKING_UPSTAIRS',
               'WALKING_DOWNSTAIRS',
               'SITTING',
               'STANDING',
               'LAYING']

# 예측 결과를 저장할 리스트
predictions = []
indices = []

logger.info("Starting prediction on MergeData.csv...")
# MergeData.csv를 청크 단위로 로드하여 예측 수행
reader = pd.read_csv(merge_data_path,
                     chunksize=chunk_size,
                     names=column_names,
                     header=0)

for chunk in tqdm(reader, desc="Predicting MergeData"):
    # 자이로스코프 데이터 추출 (X, Y, Z 컬럼)
    gyro_data = chunk[['X', 'Y', 'Z']].values

    if gyro_data.shape[0] < window_size:
        continue  # 데이터 길이가 윈도우 크기보다 작으면 건너뜀

    # 슬라이딩 윈도우 적용
    windows = sliding_window(gyro_data, window_size, step_size)

    if windows.size == 0:
        continue

    # 특성 추출
    features = extract_features(windows)

    # 데이터 증강 적용 (레이블이 없으므로 증강된 데이터의 레이블은 -1로 설정)
    # 여기서는 증강된 데이터를 사용하지 않고 원본 데이터만 예측합니다.

    # 시퀀스 생성
    X_seq, _ = create_sequences(features, np.zeros(len(features)), seq_len)

    if X_seq.size == 0:
        continue

    # 모델 입력을 위한 텐서 변환
    X_tensor = torch.tensor(X_seq, dtype=torch.float32).to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(X_tensor)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

        # 각 시퀀스의 시작 인덱스를 저장
        window_starts = np.arange(0, len(features) - seq_len + 1)
        indices.extend(chunk.index.values[window_starts])

logger.info("Prediction on MergeData.csv 완료.")

## 14.2. 예측된 레이블 추가 및 저장

In [ ]:
logger.info("Adding predicted labels to MergeData and saving the results...")

# 예측된 활동 이름으로 변환
predicted_activities = [class_names[pred] for pred in predictions]

# 결과를 DataFrame으로 저장
result_df = pd.DataFrame({
    'Index': indices,
    'Predicted Activity': predicted_activities
})

# 원본 MergeData에서 해당 인덱스의 데이터 추출
merge_data_full = pd.read_csv(merge_data_path, names=column_names, header=0)
predicted_data = merge_data_full.iloc[indices].copy()
predicted_data['Activity'] = predicted_activities

# 결과를 CSV 파일로 저장
predicted_data.to_csv('MergeData_Prediction.csv', index=False)
logger.info("예측 결과를 'MergeData_Prediction.csv'에 저장했습니다.")

## 14.3. 예측 결과 시각화

In [ ]:
logger.info("Visualizing the distribution of predicted activities...")

# 예측된 활동 레이블의 분포 시각화
activity_counts = pd.Series(predicted_activities).value_counts()
plt.figure(figsize=(8, 6))
sns.barplot(x=activity_counts.index, y=activity_counts.values, palette='viridis')
plt.title('Predicted Activity Distribution')
plt.xlabel('Activity')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 15. 마무리

In [ ]:
logger.info("모델 학습 및 예측 과정이 완료되었습니다.")
logger.info("모델과 스케일러, PCA 모델을 저장합니다.")

# 최종 모델 저장 (이미 최적의 모델은 'best_model.pth'로 저장됨)
torch.save(model.state_dict(), 'final_model.pth')
logger.info("최종 모델을 'final_model.pth'로 저장했습니다.")

# 스케일러 저장 (이미 저장됨)
# joblib.dump(scaler, 'scaler_total.pkl')
# logger.info("스케일러를 'scaler_total.pkl'로 저장했습니다.")

logger.info("전체 프로세스가 성공적으로 완료되었습니다.")